In [1]:
import os
SCALA_VERSION = '2.12'
SPARK_VERSION = '3.1.3'

os.environ['PYSPARK_SUBMIT_ARGS'] = f'--packages org.apache.spark:spark-sql-kafka-0-10_{SCALA_VERSION}:{SPARK_VERSION} pyspark-shell'

In [2]:
import findspark
findspark.init()

In [3]:
import pyspark

from pyspark.sql import SparkSession

spark = SparkSession.builder.master("local[1]")\
                            .config('spark.sql.shuffle.partitions', 4)\
                            .appName("Sparkusecase1min").getOrCreate()

22/04/01 10:40:32 WARN Utils: Your hostname, ubuntu-virtual-machine resolves to a loopback address: 127.0.1.1; using 192.168.80.128 instead (on interface ens33)
22/04/01 10:40:32 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


:: loading settings :: url = jar:file:/opt/spark-3.1.3-bin-hadoop2.7/jars/ivy-2.4.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/ubuntu/.ivy2/cache
The jars for the packages stored in: /home/ubuntu/.ivy2/jars
org.apache.spark#spark-sql-kafka-0-10_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-9f59fec6-23a0-4bb2-8cd1-d7615af70bb0;1.0
	confs: [default]
	found org.apache.spark#spark-sql-kafka-0-10_2.12;3.1.3 in central
	found org.apache.spark#spark-token-provider-kafka-0-10_2.12;3.1.3 in central
	found org.apache.kafka#kafka-clients;2.6.0 in central
	found com.github.luben#zstd-jni;1.4.8-1 in central
	found org.lz4#lz4-java;1.7.1 in central
	found org.xerial.snappy#snappy-java;1.1.8.2 in central
	found org.slf4j#slf4j-api;1.7.30 in central
	found org.spark-project.spark#unused;1.0.0 in central
	found org.apache.commons#commons-pool2;2.6.2 in central
:: resolution report :: resolve 695ms :: artifacts dl 14ms
	:: modules in use:
	com.github.luben#zstd-jni;1.4.8-1 from central in [default]
	org.apache.commons#commons-pool2;2.6.2 from central

In [8]:
kafkaDf = spark.readStream.format("kafka")\
  .option("kafka.bootstrap.servers", "localhost:9092")\
  .option("subscribe", "stock-ticks")\
  .option("group.id", "stock-ticks-group-pj-usecase3")\
   .option("timestampFormat", "dd-MMM-yyyy hh:mm")\
  .load()

In [9]:
kafkaDf.printSchema()

root
 |-- key: binary (nullable = true)
 |-- value: binary (nullable = true)
 |-- topic: string (nullable = true)
 |-- partition: integer (nullable = true)
 |-- offset: long (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- timestampType: integer (nullable = true)



In [10]:
ticksDf = kafkaDf.selectExpr("CAST(value AS STRING)", "timestamp")
ticksDf.printSchema()

root
 |-- value: string (nullable = true)
 |-- timestamp: timestamp (nullable = true)



In [11]:
import pyspark.sql.functions as F
from pyspark.sql.types import StructField,StructType,DoubleType,StringType,LongType,TimestampType

schema = StructType([
    StructField("symbol",StringType(),True),
    StructField("price",DoubleType(),True),
    StructField("volume",LongType(),True),
    StructField("timestamp",LongType(),True),

])

In [12]:
jsonDf = ticksDf.withColumn("value",F.from_json("value",schema))
jsonDf.printSchema()

root
 |-- value: struct (nullable = true)
 |    |-- symbol: string (nullable = true)
 |    |-- price: double (nullable = true)
 |    |-- volume: long (nullable = true)
 |    |-- timestamp: long (nullable = true)
 |-- timestamp: timestamp (nullable = true)



In [13]:
stockTickDf = jsonDf.select(F.col("value.*"))
stockTickDf.printSchema()

root
 |-- symbol: string (nullable = true)
 |-- price: double (nullable = true)
 |-- volume: long (nullable = true)
 |-- timestamp: long (nullable = true)



In [14]:
stockTickDf = stockTickDf.withColumn("traded_value",F.col("price")* F.col("volume"))
stockTickDf.printSchema()

root
 |-- symbol: string (nullable = true)
 |-- price: double (nullable = true)
 |-- volume: long (nullable = true)
 |-- timestamp: long (nullable = true)
 |-- traded_value: double (nullable = true)



In [15]:
stockTickDf = stockTickDf.withColumn("timestampTemp",(F.col("timestamp")/1000).cast("timestamp"))\
                    .withColumn("trade_time",F.date_trunc("minute",F.col("timestampTemp")))\
                    .drop("timestamp")\
                    .drop("timestampTemp")\
                    .withColumnRenamed("trade_time","timestamp")


In [16]:
stockTickDf1Min = stockTickDf\
                           .groupBy("symbol", F.window("timestamp", "60 seconds"))\
                          .agg( F.sum("volume").alias("volume"),\
                                F.sum("traded_value").alias("traded_value"),\
                                F.max("price").alias("high"),\
                                F.min("price").alias("low"),\
                                F.first("price").alias("first"),\
                                F.last("price").alias("last")\
                              )
stockTickDf1Min.printSchema()


stockTickDf1MinKafka = stockTickDf1Min\
                            .selectExpr("to_json(struct(*)) AS value")

stockTickDf1MinKafka.printSchema()

root
 |-- symbol: string (nullable = true)
 |-- window: struct (nullable = false)
 |    |-- start: timestamp (nullable = true)
 |    |-- end: timestamp (nullable = true)
 |-- volume: long (nullable = true)
 |-- traded_value: double (nullable = true)
 |-- high: double (nullable = true)
 |-- low: double (nullable = true)
 |-- first: double (nullable = true)
 |-- last: double (nullable = true)

root
 |-- value: string (nullable = true)



In [17]:
stockTickDf3Min = stockTickDf\
                           .groupBy("symbol", F.window("timestamp", "3 minutes"))\
                          .agg( F.sum("volume").alias("volume"),\
                                F.sum("traded_value").alias("traded_value"),\
                                F.max("price").alias("high"),\
                                F.min("price").alias("low"),\
                                F.first("price").alias("first"),\
                                F.last("price").alias("last")\
                              )
stockTickDf3Min.printSchema()


stockTickDf3MinKafka = stockTickDf3Min\
                            .selectExpr("to_json(struct(*)) AS value")


root
 |-- symbol: string (nullable = true)
 |-- window: struct (nullable = false)
 |    |-- start: timestamp (nullable = true)
 |    |-- end: timestamp (nullable = true)
 |-- volume: long (nullable = true)
 |-- traded_value: double (nullable = true)
 |-- high: double (nullable = true)
 |-- low: double (nullable = true)
 |-- first: double (nullable = true)
 |-- last: double (nullable = true)



In [ ]:
stockTickDf5Min = stockTickDf\
                           .groupBy("symbol", F.window("timestamp", "5 minutes"))\
                          .agg( F.sum("volume").alias("volume"),\
                                F.sum("traded_value").alias("traded_value"),\
                                F.max("price").alias("high"),\
                                F.min("price").alias("low"),\
                                F.first("price").alias("first"),\
                                F.last("price").alias("last")\
                              )
stockTickDf5Min.printSchema()


stockTickDf5MinKafka = stockTickDf5Min\
                            .selectExpr("to_json(struct(*)) AS value")

In [ ]:
stockTickDf1MinKafka.writeStream.format("kafka")\
                    .outputMode("complete")\
                     .option("kafka.bootstrap.servers", "localhost:9092")\
                    .option("topic", "tick1min")\
                    .option("checkpointLocation", "file:///tmp/spark12")\
                    .start()
                    

In [ ]:
stockTickDf3MinKafka.writeStream.format("kafka")\
                    .outputMode("complete")\
                     .option("kafka.bootstrap.servers", "localhost:9092")\
                    .option("topic", "tick-data3min")\
                    .option("checkpointLocation", "file:///tmp/spark32")\
                    .start()
                    

In [ ]:
stockTickDf5MinKafka.writeStream.format("kafka")\
                    .outputMode("complete")\
                     .option("kafka.bootstrap.servers", "localhost:9092")\
                    .option("topic", "tick-data5min")\
                    .option("checkpointLocation", "file:///tmp/spark52")\
                    .start()
                    